#2.미션 수행

##1.환경 설정 및 라이브러리 설치
LangChain, Hugging Face 모델, 벡터 DB 사용을 위한 필수 라이브러리를 설치합니다.

In [2]:
# 1. LangChain 핵심 모듈 및 커뮤니티 모듈 설치 (최신 표준)
!pip install langchain  # 메타 패키지 (langchain-core, langchain-community 포함)

# 2. PDF 문서 로더 (PyPDF)
!pip install pypdf

# 3. 벡터 데이터베이스 (FAISS)
!pip install faiss-cpu
#!pip install langchain-community  # (혹시모를) 벡터스토어/로더 등을 위한 커뮤니티 패키지 보장
#!pip install -U --force-reinstall langchain langchain-community langchain-core langchain-text-splitters # Upgrade all to ensure consistency
!pip install langchain_community langchain_text_splitters langchain_openai langchain_chroma

# 4. 임베딩 모델 (Hugging Face Sentence Transformers)
!pip install sentence-transformers

# 5. LLM 구동 및 양자화 (Hugging Face Models)
!pip install transformers accelerate bitsandbytes
!pip install torch # PyTorch가 설치되어 있어야 함 (Colab 기본 설치)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 116.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 kB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 104.5 MB/s eta 0:0

In [3]:
import os
import requests

import torch
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from transformers import pipeline

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms import HuggingFacePipeline
from langchain_community.document_loaders import PyPDFLoader, TextLoader

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

##2.문서 로드 및 청킹 (Preprocessing)
국세청 PDF 문서를 로드하고, 검색 효율을 위해 적절한 크기로 분할합니다.

###2.1.문서 로드
LangChain의 PyPDFLoader를 사용합니다. 데이터셋은 '국세청 연말정산 종합 안내 페이지'에서 PDF 파일을 다운로드하여 사용합니다.

In [4]:
#구글드라이브에 업로드
from google.colab import files
uploaded = files.upload()

Saving (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업.pdf to (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업.pdf


In [5]:
#업로드한 문서파일 확인
for fn in uploaded.keys():
	print('User uploaded file "{name}" with {length} bytes'.format(name=fn, length=len(uploaded[fn])))

User uploaded file "(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업.pdf" with 2377353 bytes


In [7]:
# ==========================================
# 0. 초기 설정
# ==========================================
import os
from getpass import getpass

# OpenAI API key
openai_api_key = getpass("Enter your OpenAI API key: ")

# API키 설정
os.environ["OPENAI_API_KEY"] = openai_api_key

print('You Entered OpenAI API key')

Enter your OpenAI API key: ··········
You Entered OpenAI API key


In [8]:
import re
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# 1. 문서 로드
pdf_path = "/content/(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()

# 2. 텍스트 전처리 (노이즈 제거)
for doc in documents:
    # 페이지 번호, 과도한 공백, 목차 점 등을 제거
    doc.page_content = re.sub(r'^\s*-\s*\d+\s*-\s*$', '', doc.page_content, flags=re.MULTILINE)
    doc.page_content = re.sub(r'·{2,}', '', doc.page_content)
    doc.page_content = re.sub(r'\s+', ' ', doc.page_content).strip()

# 3. 문서 분할 (Chunk Size 대폭 상향)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,    # 핵심 수정 사항
    chunk_overlap=200,  # 핵심 수정 사항
    separators=["\n\n", "\n", " ", ""]
)
splits = text_splitter.split_documents(documents)
print(f"분할된 청크 개수: {len(splits)}")

# 4. 임베딩 및 벡터 저장소 생성
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(model="text-embedding-3-small")
)

# 5. 검색기 및 체인 생성 (k값 상향)
retriever = vectorstore.as_retriever(search_kwargs={"k": 6})
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

template = """
당신은 벤처기업협회의 담당 AI입니다.
아래 [규정]을 참고하여 질문에 친절하고 상세하게 답해주세요.
문서에 없는 내용은 "문서에 나와있지 않습니다"라고 답하세요.

[규정]
{context}

[질문]: {question}
"""
prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 6. 질문 테스트
response = rag_chain.invoke("벤처확인종합시스템 기능 고도화에 대해서 상세하게 알려 줘.")
print(response)

분할된 청크 개수: 175
벤처확인종합관리시스템 기능 고도화 사업은 2024년에 진행되는 프로젝트로, 주로 복수의결권주식, 스톡옵션, 성과조건부주식의 교부 기능을 구축하고 고도화하는 것을 목표로 하고 있습니다. 아래는 이 사업의 주요 내용입니다.

### 1. 사업 개요
- **사업명**: 2024년 벤처확인종합관리시스템 기능 고도화
- **사업기간**: 계약일로부터 150일
  - **1차 오픈**: 2024년 7월 10일 (성과조건부 및 스톡옵션 오픈)
  - **2차 오픈**: 2024년 9월 중 Grand Open
- **소요예산**: 352,000,000원 (부가가치세 포함)
- **계약방식**: 제한경쟁입찰(협상에 의한 계약)

### 2. 추진 목표
- 벤처확인 연계를 통한 복수의결권 및 스톡옵션 시스템의 고도화
- 복수의결권주식 및 스톡옵션 업무 시스템 구축
- 스톡옵션 데이터 이관 작업 수행
- 신청 접수 및 처리 결과 알림을 위한 One-Stop Service 추진

### 3. 세부 실행 내용
- **대민사이트**:
  - 비상장 벤처확인기업 조회, 복수의결권주식 발행·변경 보고 고도화, 스톡옵션 부여, 취소·철회 신고 기능 구축
  - 단계별 처리 절차 준수
  - 복수의결권 및 스톡옵션 정보 확인 기능 구축

- **BackOffice**:
  - 감독기관 전용 시스템 구축
  - 복수의결권주식 및 스톡옵션 관리 기능
  - 통계 및 추가 사항 관리
  - 타 기관 API 연계를 통한 정보 확인
  - 입력 정보 및 서류 제출 간소화, 정보 보안 솔루션 적용

### 4. 제안 요청 일반사항
- 제안 업체는 시스템 기능 고도화 완료 후 기능 확장성을 고려하여 제안해야 하며, 모든 부분에 포괄적인 책임을 져야 합니다.
- 전산 시스템 운용 및 유지관리에 대한 구체적인 기술 이전, 교육 계획 및 지원 계획을 제시해야 합니다.
- 시스템 개발 과정에서 취득한 모든 정보는 외부로 유출해서는 안 되며, 보안사고 발생 시 업체는 민형사상의 책임을 

In [9]:
# ==========================================
# 5. 실행 (Test)
# ==========================================
question_2 = "벤처확인종합시스템 기능 고도화의 사업개요에 대해서 상세하게 알려 줘.."

print(f"\n 질문: {question_2}")
print("AI 답변 생성 중...\n")

response = rag_chain.invoke(question_2)

print(response)


 질문: 벤처확인종합시스템 기능 고도화의 사업개요에 대해서 상세하게 알려 줘..
AI 답변 생성 중...

벤처확인종합관리시스템 기능 고도화의 사업개요는 다음과 같습니다:

1. **사업명**: 2024년 「벤처확인종합관리시스템 기능 고도화」 용역사업
   - 이 사업은 복수의결권주식, 스톡옵션, 성과조건부주식 교부 기능을 구축하고 고도화하는 것을 목표로 합니다.

2. **사업기간**: 계약일로부터 150일
   - 1차 오픈: 2024년 7월 10일 (성과조건부 및 스톡옵션 오픈)
   - 2차 오픈: 2024년 9월 중 Grand Open

3. **소요예산**: 352,000,000원 (부가가치세 포함)

4. **계약방식**: 제한경쟁입찰(협상에 의한 계약)
   - 이 계약은 「국가를 당사자로 하는 계약에 관한 법률」 시행령 제21조, 제43조에 근거하여 진행됩니다.

5. **선정 방법**: 
   - 본 사업은 전문기술이 요구되는 사업으로, 기술력이 높은 업체를 선정하고 협상을 통해 요구사항을 반영할 수 있는 '협상에 의한 계약' 방식으로 추진됩니다.
   - 제안서 평가는 기술평가(90%)와 가격평가(10%)를 합산하여 종합평가점수로 산출되며, 배정한도 85% 이상을 고득점 순으로 협상 적격자를 결정합니다.

6. **대기업 참여 제한**: 
   - 「소프트웨어 진흥법」 제48조 제2항에 따라, 사업 금액 20억원 미만 계약은 대기업의 참여가 제한됩니다.

이와 같은 내용이 벤처확인종합관리시스템 기능 고도화 사업의 개요입니다. 추가적인 질문이 있으시면 언제든지 말씀해 주세요!


In [10]:
# ==========================================
# 5. 실행 (Test)
# ==========================================
question_3 = "벤처확인종합시스템 기능 고도화의 사업기간에 대해서 상세하게 알려 줘.."

print(f"\n 질문: {question_3}")
print("AI 답변 생성 중...\n")

response = rag_chain.invoke(question_3)

print(response)


 질문: 벤처확인종합시스템 기능 고도화의 사업기간에 대해서 상세하게 알려 줘..
AI 답변 생성 중...

벤처확인종합관리시스템 기능 고도화의 사업기간은 계약일로부터 150일입니다. 이 기간 동안 두 차례의 오픈이 예정되어 있습니다. 

1. **1차 오픈**: 2024년 7월 10일에 성과조건부 및 스톡옵션 기능이 오픈됩니다.
2. **2차 오픈**: 2024년 9월 중에 Grand Open이 예정되어 있습니다. 

이와 같은 일정에 따라 사업이 진행될 예정입니다.
